In [1]:
import dace
import cupy as cp

In [2]:
N = dace.symbol("N")
M = dace.symbol("M")
J = dace.symbol("J")
K = dace.symbol("K")

sN = 7
sM = 2
sJ = 5
sK = 8

@dace.program
def vector_copy_strides4d(A: dace.uint32[N, M, J, K] @ dace.dtypes.StorageType.GPU_Global, B: dace.uint32[N, M, J, K] @ dace.dtypes.StorageType.GPU_Global):
    for a, b, c, d in dace.map[0:N:sN, 0:M:sM, 0:J:sJ, 0:K:sK] @ dace.dtypes.ScheduleType.GPU_Device:
        A[a, b, c, d] = B[a, b, c, d]

sdfg = vector_copy_strides4d.to_sdfg()
sdfg


SDFG (vector_copy_strides4d)

In [3]:
# Example input 
n = 12
m = 14
j = 11
k = 23

B = cp.ones((n, m, j, k, ), dtype=cp.uint32)
A = cp.zeros((n, m, j, k, ), dtype=cp.uint32)


# Strided copy from B to A
sdfg(A=A, B=B, N=n, M=m, J=j, K=k)



# Verify correctness for 2D strided copy
expected = cp.zeros((n, m, j, k, ), dtype=cp.uint32)
expected[::sN, ::sM, ::sJ, ::sK] = 1
if (A == expected).all():
    print("\n\nSUCCESS: A matches the expected result.")
else:
    print("\n\nERROR: A does not match the expected result.")



SUCCESS: A matches the expected result.
